In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('../input/placement-visualization-prediction/datasets_placememt.csv')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.info()

# Description of this Dataset

1. sl_no -Serial Number
2. Gender- Male='M',Female='F'
3. ssc_p-Secondary Education percentage- 10th Grade 
4. ssc_b-Board of Education- Central/ Others
5. hsc_pHigher Secondary Education percentage- 12th Grade
6. hsc_b-Board of Education- Central/ Others
7. hsc_s-Specialization in Higher Secondary Education 
8. degree_p-Degree Percentage
9. degree_t-Under Graduation(Degree type)- Field of degree educatio
10. workex-Work Experience
11. etest_p = Employability test percentage ( conducted by college)
12. specialisation = Post Graduation(MBA)- Specialization
13. mba_p = MBA percentage
14. status = Status of placement- Placed/Not placed
15. salary = Salary offered by corporate to candidates

In [ ]:
#Check the datatypes of this dataset
data.dtypes

In [ ]:
#Check the null values
data.isnull().sum()

In [ ]:
data.describe(include='all')

In [ ]:
#clean the nan value
data['salary'].fillna(data['salary'].mean(), inplace=True)

In [ ]:
#after cleaning the nan value check the null value
data.isnull().sum()

Now my dataset has fully cleaned

# Analyzing the datase

In [ ]:
sns.countplot(data.status)
plt.show()

We can see that 140 above students ara placed and nearby 80 are not placed

In [ ]:
data['status'].value_counts()

In [ ]:
sns.countplot(data.gender,hue=data.status)
plt.show()

We can see that most of the Mens are placed in this placement

In [ ]:
sns.distplot(data.ssc_p[data.status=='Not Placed'])
sns.distplot(data.ssc_p[data.status=='Placed'])
plt.legend(['Not placed','placed'])
plt.show()

In [ ]:
data['ssc_b'].value_counts()

In [ ]:
sns.countplot(data.ssc_b,hue=data.status)
plt.show()

In [ ]:
sns.distplot(data.hsc_p[data.status=='Not Placed'])
sns.distplot(data.hsc_p[data.status=='Placed'])
plt.legend(['Not placed','placed'])
plt.show()

In [ ]:
data['hsc_b'].value_counts()

In [ ]:
sns.countplot(data.hsc_b,hue=data.status)
plt.show()

In [ ]:
data['hsc_s'].value_counts()

In [ ]:
sns.countplot(data.hsc_s,hue=data.status)
plt.show()

In [ ]:
sns.distplot(data.degree_p[data.status=='Not Placed'])
sns.distplot(data.degree_p[data.status=='Placed'])
plt.legend(['Not placed','placed'])
plt.show()

In [ ]:
data['degree_t'].value_counts()

In [ ]:
sns.countplot(data.degree_t,hue=data.status)
plt.show()

In [ ]:
data['workex'].value_counts()

In [ ]:
sns.countplot(data.workex,hue=data.status)
plt.show()

In [ ]:
sns.distplot(data.etest_p[data.status=='Not Placed'])
sns.distplot(data.etest_p[data.status=='Placed'])
plt.legend(['Not placed','placed'])
plt.show()

In [ ]:
data['specialisation'].value_counts()

In [ ]:
sns.countplot(data.specialisation,hue=data.status)
plt.show()

In [ ]:
sns.distplot(data.mba_p[data.status=='Not Placed'])
sns.distplot(data.mba_p[data.status=='Placed'])
plt.legend(['Not placed','placed'])
plt.show()

In [ ]:
sns.swarmplot(x=data.salary,y=data.gender,hue=data.status,data=data)
plt.show()

In [ ]:
#drop the unrelevent column
data.drop(['sl_no'],axis=1,inplace=True)

# LOGISTIC REGRESSION


* Taitanic dataset contain text or categorical values
* To make the data understandable or in human readable form, the training data is often labeled in words.
* Label Encoding refers to converting the labels into numeric form so as to convert it into the machine-readable form.

In [ ]:
#Label encoder
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
data.gender=lb.fit_transform(data.gender)

le1=LabelEncoder()
data.ssc_b=le1.fit_transform(data.ssc_b)

lb2=LabelEncoder()
data.hsc_b=lb2.fit_transform(data.hsc_b)

le3=LabelEncoder()
data.hsc_s=le3.fit_transform(data.hsc_s)

lb4=LabelEncoder()
data.degree_t=lb4.fit_transform(data.degree_t)

le5=LabelEncoder()
data.workex=le5.fit_transform(data.workex)

lb6=LabelEncoder()
data.specialisation=lb6.fit_transform(data.specialisation)

le7=LabelEncoder()
data.status=le7.fit_transform(data.status)

In [ ]:
data

# Pearson's Correlation

In [ ]:
corr=data.corr()
plt.figure(figsize=(12,5))
sns.heatmap(corr,annot=True,cmap='coolwarm')
plt.show()

In [ ]:
#input and output selection
ip=data.drop(['status'],axis=1)
op=data['status']

# One-Hot Encoing

* a one-hot encoding can be applied to the integer representation. This is where the integer encoded      variable is removed and a new binary variable is added for each unique integer value.

* In the “color” variable example, there are 3 categories and therefore 3 binary variables are needed. A “1” value is placed in the binary variable for the color and “0” values for the other colors.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([('status',OneHotEncoder(),[5,7])],remainder='passthrough')

ip = np.array(ct.fit_transform(ip),dtype = np.str)

# Train and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts=train_test_split(ip,op,test_size=0.1)

# Scalling our dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(xtr)
xtr=sc.transform(xtr)
xts=sc.transform(xts)

In [ ]:
from sklearn.linear_model import LogisticRegression
alg=LogisticRegression()

In [ ]:
#train the algorithm with the training data
alg.fit(xtr,ytr)
yp=alg.predict(xts)

# checking accuracy of the model


In [ ]:
from sklearn import metrics
cm=metrics.confusion_matrix(yts,yp)
print(cm)

In [ ]:
accuracy=metrics.accuracy_score(yts,yp)
print(accuracy)

In [ ]:
precission=metrics.precision_score(yts,yp)
print(precission)

In [ ]:
recall=metrics.recall_score(yts,yp)
print(recall)